<a href="https://colab.research.google.com/github/Elwing-Chou/ml0716/blob/master/transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import zipfile
f = zipfile.ZipFile("drive/My Drive/additional/train.zip")
f.extractall("datas")

In [6]:
import glob
import pandas as pd
dogs = glob.glob("datas/train/dog.*")
cats = glob.glob("datas/train/cat.*")
df = pd.DataFrame({
    "path":dogs + cats,
    "ans":[0] * len(dogs) + [1] * len(cats)
})
df

,path,ans
0,datas/train/dog.11834.jpg,0
1,datas/train/dog.2311.jpg,0
2,datas/train/dog.4547.jpg,0
3,datas/train/dog.4702.jpg,0
4,datas/train/dog.5517.jpg,0
...,...,...
24995,datas/train/cat.325.jpg,1
24996,datas/train/cat.8744.jpg,1
24997,datas/train/cat.8227.jpg,1
24998,datas/train/cat.3499.jpg,1
